In [5]:
import pandas as pd
from urllib.request import urlopen
import json

# Set Tickers and Columns

tickers for all the current S&P500 stocks are preloaded into tickers.csv. I've selected all the available columns from the security quotes.

In [6]:
tickers = list(pd.read_csv("tickers.csv").Ticker)

In [7]:
requests = ["income-statement"]

#### DATA PARSER
the following function will parse a .JSON and output as a list of python dictionaries

In [8]:
def get_jsonparsed_data(url)->dict:

    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

#### CSV CONVERTER
the following function will convert the dictionaries to .csv files

In [70]:
def to_csv(json_dat,req,tick):
    data ={}
    cols = list(json_dat[0].keys())
    for col in cols:
        data[col] = []
        
    for security in json_dat:
        for key,val in security.items():
            data[key].append(val)
            
    df = pd.DataFrame(data)
    
    df.to_csv("{}/SP500-{}-{}.csv".format(req,tick,req))

# Pull Data
We'll be getting data from the financialmodelingprep.com API. Usage requires registration, but you can make a free account with up to 250 requests per day. If you want to use this code make sure to create folders in your working directory for 'company-key-metrics' and 'cash-flow-statement'. As we need make around 700 requests, you'll need three API keys (or if you can afford it use a premium API)

In [83]:
API_KEY1 = "xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx"
API_KEY2 = "xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx"
API_KEY3 = "85f03f84ead0af595e31643a3ad50f7c"
url_temp = "https://financialmodelingprep.com/api/v3/{}/{}?apikey={}"

#### COMPANY KEY METRICS

In [82]:
idx = 0
print("FETCHING COMPANY KEY METRICS:")
while idx < len(tickers):
    
    end_idx = idx+3 if idx+3<len(tickers) else len(tickers)
    
    print("{}/{} ---> FETCHING {}".format(end_idx,len(tickers),tickers[idx:end_idx]))
    url = url_temp.format("company-key-metrics",",".join(tickers[idx:end_idx]),API_KEY3)
    print(url)
   
    stream = get_jsonparsed_data(url)
    
    try:
        for dictionary in stream["keyMetricsList"]:
            to_csv(dictionary["metrics"],"company-key-metrics",dictionary['symbol'])
    except:
        try:
            to_csv(stream["metrics"],"company-key-metrics",stream['symbol'])
        except:
            print("ERROR -> {}".format(tickers[idx:end_idx]))
            print("STREAM -> {}".format(stream))
            
        
    idx+=3




FETCHING COMPANY KEY METRICS:
3/505 ---> FETCHING ['MMM', 'ABT', 'ABBV']
https://financialmodelingprep.com/api/v3/company-key-metrics/MMM,ABT,ABBV?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> ['MMM', 'ABT', 'ABBV']
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}
6/505 ---> FETCHING ['ABMD', 'ACN', 'ATVI']
https://financialmodelingprep.com/api/v3/company-key-metrics/ABMD,ACN,ATVI?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> ['ABMD', 'ACN', 'ATVI']
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}
9/505 ---> FETCHING ['ADBE', 'AMD', 'AAP']
https://financialmodelingprep.com/api/v3/company-key-metrics/ADBE,AMD,AAP?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> ['ADBE', 'AMD', 'AAP']
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentat

KeyboardInterrupt: 

#### CASHFLOWS

In [80]:
for idx in range(0,len(tickers)):
    if idx<249:
        API_KEY = API_KEY1
    elif idx<498:
        API_KEY = API_KEY2
    else:
        API_KEY = API_KEY3
            
            
    tick = tickers[idx]
    print("TICKER:{} ---> {}/{}".format(tick,idx+1,len(tickers)))
    url = url_temp.format("cash-flow-statement",tick,API_KEY)
    print(url)
    stream = get_jsonparsed_data(url)
    try:
        to_csv(stream,"cash-flow-statement",tick)
    except:
        print("ERROR -> {}".format(tick))
        print("STREAM -> {}".format(stream))
    

TICKER:MMM ---> 1/505
https://financialmodelingprep.com/api/v3/cash-flow-statement/MMM?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> MMM
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}
TICKER:ABT ---> 2/505
https://financialmodelingprep.com/api/v3/cash-flow-statement/ABT?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> ABT
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}
TICKER:ABBV ---> 3/505
https://financialmodelingprep.com/api/v3/cash-flow-statement/ABBV?apikey=xxxx-xxxx-xxxx-xxxx-xxxx-xxxx-xxxx
ERROR -> ABBV
STREAM -> {'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}
TICKER:ABMD ---> 4/505
https://financialmodelingprep.com/api/v3/cash-flow-statement/ABMD?apikey=xxxx-xx

KeyboardInterrupt: 

 **N.B** there are a few missing tickers for cash-flows, namely BF.B and BRK.B

#### PROFILES

In [84]:
 url = url_temp.format("profile",",".join(tickers),API_KEY3)
stream = get_jsonparsed_data(url)
stream.sort(key = lambda x: x['symbol'])
data ={}
cols = list(stream[0].keys())
for col in cols:
    data[col] = []
        
for security in stream:
    for key,val in security.items():
        data[key].append(val)
            
df = pd.DataFrame(data)
df.to_csv('profiles.csv')